In [1]:
import tensorflow.keras.applications as models 

from libsvm.svmutil import svm_train, svm_problem, svm_parameter

from matplotlib import pyplot

mdls = {}
filters = {}
biases = {}

mdls['vgg16'] = models.vgg16.VGG16()
mdls['mob224'] = models.mobilenet.MobileNet()
mdls['inc_v2'] = models.inception_resnet_v2.InceptionResNetV2()
mdls['eff_b2'] = models.efficientnet.EfficientNetB4()

def extr_mdl_ker(mdls):
    for idx, mdl in enumerate(mdls):
        flag = True 
        l_idx = 0
        mdl_name = f"{mdl}" 
        layers = mdls[mdl_name].layers[:10]
        for l_idx, layer in enumerate(layers):
            if 'conv' in layer.name and 'pad' not in layer.name:
                flag = False
                break 
            if not flag:
                break
        ker_weight = mdls[mdl_name].layers[l_idx].get_weights()
        # print(ker_weight)
        if len(ker_weight) == 1:
            filters[mdl_name] = ker_weight[0]
        else:
            filters[mdl_name], _ = ker_weight
        print(mdl_name, ' layer:', l_idx, layer.name, ' shape', filters[mdl_name].shape)
        
    return filters


filters = extr_mdl_ker(mdls)
    
    
# vgg16_filters, vgg16_biases = mdls['vgg16'].layers[1].get_weights()



vgg16  layer: 1 block1_conv1  shape (3, 3, 3, 64)
mob224  layer: 1 conv1  shape (3, 3, 3, 32)
inc_v2  layer: 1 conv2d  shape (3, 3, 3, 32)
eff_b2  layer: 4 stem_conv  shape (3, 3, 3, 48)


In [42]:
for i, layer in enumerate(mdls['vgg16'].layers[:20]):
    if 'conv' not in layer.name or ('pad' in layer.name):
        continue
    print(layer.name, 'index', i)
    print(type(layer.name))

block1_conv1 index 1
<class 'str'>
block1_conv2 index 2
<class 'str'>
block2_conv1 index 4
<class 'str'>
block2_conv2 index 5
<class 'str'>
block3_conv1 index 7
<class 'str'>
block3_conv2 index 8
<class 'str'>
block3_conv3 index 9
<class 'str'>
block4_conv1 index 11
<class 'str'>
block4_conv2 index 12
<class 'str'>
block4_conv3 index 13
<class 'str'>
block5_conv1 index 15
<class 'str'>
block5_conv2 index 16
<class 'str'>
block5_conv3 index 17
<class 'str'>


In [2]:
r50 = models.resnet50.ResNet50()

for i, layer in enumerate(r50.layers[:20]):
    print(layer.name)


input_5
conv1_pad
conv1_conv
conv1_bn
conv1_relu
pool1_pad
pool1_pool
conv2_block1_1_conv
conv2_block1_1_bn
conv2_block1_1_relu
conv2_block1_2_conv
conv2_block1_2_bn
conv2_block1_2_relu
conv2_block1_0_conv
conv2_block1_3_conv
conv2_block1_0_bn
conv2_block1_3_bn
conv2_block1_add
conv2_block1_out
conv2_block2_1_conv
